# Reading 17: Regular Expressions & Grammars (JAVA)
Objectives
After today’s class, you should:

* Understand the ideas of grammar productions and regular expression operators
* Be able to read a grammar or regular expression and determine whether it matches a sequence of characters
* Be able to write a grammar or regular expression to match a set of character sequences and parse them into a data structure

今天的阅读介绍了几个概念：

* 语法，包括生成、非终结符、终结符和运算符(grammars, with productions, nonterminals, terminals, and operator)
* 正则表达式
有些程序模块以字节序列或字符序列的形式接收输入或产生输出，如果只是存储在内存中，则称为字符串，如果流入或流出模块，则称为流。在今天的阅读中，我们将讨论如何编写这样一个序列的规范。具体来说，字节或字符序列可以是

* 一个字符串
* 磁盘上的文件，在这种情况下，规范被称为文件格式
* 通过网络发送的信息，在这种情况下，the specification is a wire protocol
* 用户在控制台上键入的命令，在这种情况下，he specification is a command line interface

对于这些类型的序列，我们引入了grammar的概念，它不仅能让我们区分合法序列和非法序列，还能将序列解析为程序可以使用的数据结构。grammar产生的数据结构通常是递归数据类型，就像我们在递归数据类型阅读中谈到的那样。

我们还将讨论语法的一种特殊形式，即regular expression。除了用于规范和解析，正则表达式还是一种广泛使用的工具，可用于许多需要拆解字符串、从中提取信息或转换字符串的字符串处理任务。

下一篇阅读将讨论解析器生成器，这是一种将语法自动转化为该语法的解析器的工具。

## Grammars
要描述一串符号（无论是字节、字符还是从固定集合中抽取的其他类型符号），we use a compact representation called a grammar.
A grammar defines a set of strings.假设我们要编写一个表示 URL 的grammar。我们的 URL grammar将指定 HTTP 协议中合法 URL 的字符串集合。

grammer中的literal strings 称为terminals。它们之所以被称为terminals，是因为它们不能再进一步扩展。我们通常用引号来书写terminals，如 "http "或":"，就是常量字符。

grammar由一组production描述，每个production定义一个nonterminal。你可以把nonterminal看成是代表一组字符串的变量（实际上就是变量），而production则是该变量在其他变量（nonterminal）、运算符和常量（终端）方面的定义。非终结符是表示字符串的树的内部节点。

> To describe a string of symbols, whether they are bytes, characters, or some other kind of symbol drawn from a fixed set, we use a compact representation called a grammar.

> A grammar defines a set of strings. Suppose we want to write a grammar that represents URLs. Our grammar for URLs will specify the set of strings that are legal URLs in the HTTP protocol.

> The literal strings in a grammar are called terminals. They’re called terminals because they can’t be expanded any further. We generally write terminals in quotes, like 'http' or ':'.

> A grammar is described by a set of productions, where each production defines a nonterminal. You can think of a nonterminal like a variable that stands for a set of strings, and the production as the definition of that variable in terms of other variables (nonterminals), operators, and constants (terminals). Nonterminals are internal nodes of the tree representing a string.

> A production in a grammar has the form

grammar中的profuction形式为
> nonterminal ::= expression of terminals, nonterminals, and operators 

**Grammar 实际就是production的集合。**

**One of the nonterminals of the grammar is designated as the root**. The set of strings that the grammar recognizes are the ones that match the root nonterminal. This nonterminal is sometimes called root or start or even just S, but in the grammars below we will typically choose more readable names for the root, like url, html, and markdown.

So a grammar that represents a singleton set, allowing only one specific URL, might have just one production defining the nonterminal url, with a terminal on the righthand side:
>url ::= 'http://mit.edu/'

### Grammar operators

Productions can use operators to combine terminals and nonterminals on the righthand side. The three most important operators in a production expression are:

Repetition, represented by *:
> x ::= y*        // x matches zero or more y

Concatenation, represented not by a symbol, but just a space:
> x ::= y z       // x matches y followed by z 

Union, also called alternation, represented by |:
> x ::= y | z     // x matches either y or z 

By convention, postfix operators like * have highest precedence, which means they are applied first. Concatenation is applied next. Alternation | has lowest precedence, which means it is applied last. Parentheses can be used to override precedence:
> m ::= a (b|c) d      // m matches a, followed by either b or c, followed by d
x ::= (y z | a b)*   // x matches zero or more yz or ab pairs

Let’s use these operators to generalize our url grammar to match some other hostnames, such as http://stanford.edu/ and http://google.com/

> url ::= 'http://' hostname '/'
hostname ::= 'mit.edu' | 'stanford.edu' | 'google.com'

The first rule of this grammar demonstrates concatenation. The url nonterminal matches strings that start with the literal string http://, followed by a match to the hostname nonterminal, followed by the literal string /.

The hostname rule demonstrates union. A hostname can match one of the three literal strings, mit.edu or stanford.edu or google.com.

So this grammar represents the set of three strings, http://mit.edu/, http://google.com/, and http://stanford.edu/.

Let’s take it one step further by allowing any lowercase word in place of mit, stanford, google, com and edu:

> url ::= 'http://' hostname '/'
hostname ::= word '.' word
word ::= letter*
letter ::= ('a' | 'b' | 'c' | 'd' | 'e' | 'f' | 'g' | 'h' | 'i' 
                | 'j' | 'k' | 'l' | 'm' | 'n' | 'o' | 'p' | 'q' 
                | 'r' | 's' | 't' | 'u' | 'v' | 'w' | 'x' | 'y' | 'z')


The new word rule matches a string of zero or more lowercase letters, so the overall grammar can now match http://alibaba.com/ and http://zyxw.edu/ as well. Unfortunately word can also match an empty string, so this url grammar also matches http://./, which is not a legal URL. Here’s a verbose way to fix that, which requires word to match at least one letter.

> word ::= letter letter*


### More grammar operators
You can also use additional operators which are just syntactic sugar (i.e., they’re equivalent to combinations of the main three operators).

0 or 1 occurrence is represented by ?:

> x ::= y?       // an x is a y or is the empty string

1 or more occurrences is represented by +:

>x ::= y+       // an x is one or more y
                //    equivalent to  x ::= y y*

An exact number of occurrences is represented by {n}, and a range of occurences by {n,m}, {n,}, or {,m}:

> x ::= y{3}     // an x is three y
               // equivalent to x ::= y y y 

> x ::= y{1,3}   // an x is between one and three y
               // equivalent to x ::= y | y y | y y y

> x ::= y{,4}    // an x is at most four y
               // equivalent to x ::=   | y | y y | y y y | y y y y
               //                     ^--- note the empty string here, so this can match zero y's

> x ::= y{2,}    // an x is two or more y
               // equivalent to x ::= y y y*

A character class [...] represents the set of single-character strings matching any of the characters listed in the square brackets:

> x ::= [aeiou]  // equivalent to  x ::= 'a' | 'e' | 'i' | 'o' | 'u'

Ranges of characters can be described compactly using -:
> x ::= [a-ckx-z]    // equivalent to  x ::= 'a' | 'b' | 'c' | 'k' | 'x' | 'y' | 'z'

An inverted character class [^...] represents the set of single-character strings matching a character not listed in the brackets:

>x ::= [^a-c]  // equivalent to  x ::= 'd' | 'e' | 'f' | ... | '0' | '1' | '2' | ... | '!' | '@'
              //                          | ... (all other possible characters)


These additional operators allow the word production to be expressed more compactly:

> url ::= 'http://' hostname '/'
hostname ::= word '.' word
word ::= [a-z]+

### Recursion in grammars
How else do we need to generalize? Hostnames can have more than two components, and there can be an optional port number:

> http://didit.csail.mit.edu:4949/

> To handle this kind of string, the grammar is now:
url ::= 'http://' hostname (':' port)? '/' 
hostname ::= word '.' hostname | word '.' word
port ::= [0-9]+
word ::= [a-z]+


Notice how hostname is now defined recursively in terms of itself. Which part of the hostname definition is the base case, and which part is the recursive step? What kinds of hostnames are allowed?

Using the repetition operator, we could also write hostname without recursion, like this:

> hostname ::= (word '.')+ word

Recursion can sometimes be eliminated from a grammar using operators like this, but not always.

Another thing to observe is that this grammar allows port numbers that are not technically legal, since port numbers can only range from 0 to 65535 (216-1). We could write a more complex definition of port that would match only these integers, but that’s not typically done in a grammar. Instead, the constraint 0 ≤ port ≤ 65535 would be specified in the program that uses the grammar.

There are more things we should do to go further:

* generalizing http to support the additional protocols that URLs can have
* generalizing the / at the end to a slash-separated path
* allowing hostnames with the full set of legal characters instead of just a-z




















## Parse trees

Matching a grammar against a string can generate a parse tree that shows how parts of the string correspond to parts of the grammar.

The leaves of the parse tree are labeled with terminals, representing the parts of the string that have been parsed. They don’t have any children, and can’t be expanded any further. If we concatenate the leaves together in order, we get back the original string. A trivial example is the one-line URL grammar that we started with, whose (only possible) parse tree is shown at the right:
![](ref/lect12/2023-08-05-13-59-37.png)

> url ::= 'http://mit.edu/'



The internal nodes of the parse tree are labeled with nonterminals, since they all have children. The immediate children of a nonterminal node must follow the pattern of the nonterminal’s production rule in the grammar. For example, in our more elaborate URL grammar that allows any two-part hostname, the children of a hostname node in the tree must follow the pattern of the hostname rule, word '.' word. The figure on the right shows the parse tree produced by matching this grammar against http://mit.edu/:

> url ::= 'http://' hostname '/'
hostname ::= word '.' word
word ::= [a-z]+

![](ref/lect12/2023-08-05-14-04-09.png)

For a more elaborate example, here is the parse tree for the recursive URL grammar. The tree has more structure now. The hostname and word nonterminals are labeling nodes of the tree whose subtrees match those rules in the grammar.

>url ::= 'http://' hostname (':' port)? '/' 
hostname ::= word '.' hostname | word '.' word
port ::= [0-9]+
word ::= [a-z]+

![](ref/lect12/2023-08-05-15-04-13.png)







#### Example: Markdown and HTML
Now let’s look at grammars for some file formats. We’ll be using two different markup languages that represent typographic style in text. Here they are:

**Markdown**:

This is _italic_.

**HTML:**
Here is an <i>italic</i> word.

For simplicity, our example HTML and Markdown grammars will only specify italics, but other text styles are of course possible. Also for simplicity, we will assume the plain text between the formatting delimiters isn’t allowed to use any formatting punctuation, like _ or <.

Here’s the grammar for our simplified version of Markdown:

>markdown ::= ( normal | italic ) *
italic ::= '_' normal '_'
normal ::= text
text ::= [^_]*

![](ref/lect12/2023-08-05-15-26-03.png)

Here’s the grammar for our simplified version of HTML:

> html ::= ( normal | italic ) *
italic ::= '<i>' html '</i>'
normal ::= text
text ::= [^<>]*

![](ref/lect12/2023-08-05-15-28-35.png)

## Regular expressions
A regular grammar has a special property: by substituting every nonterminal (except the root one) with its righthand side, you can reduce it down to a single production for the root, with only terminals and operators on the right-hand side.

Our URL grammar is regular. By replacing nonterminals with their productions, it can be reduced to a single expression:

>url ::= 'http://' ([a-z]+ '.')+ [a-z]+ (':' [0-9]+)? '/' 

The Markdown grammar is also regular:
> markdown ::= ([^_]* | '_' [^_]* '_' )*

But our HTML grammar can’t be reduced completely. By substituting righthand sides for nonterminals, you can eventually reduce it to something like this:

```regex
html ::= ( [^<>]* | '<i>' html '</i>' )*
```
Unfortunately, the recursive use of html on the righthand side can’t be eliminated, and can’t be simply replaced by a repetition operator either. So the HTML grammar is not regular.

The reduced expression of terminals and operators can be written in an even more compact form, called a regular expression. A regular expression does away with the quotes around the terminals, and the spaces between terminals and operators, so that it consists just of terminal characters, parentheses for grouping, and operator characters. For example, the regular expression for our markdown format is just

```regex
([^_]*|_[^_]*_)*
```

Regular expressions are also called regexes for short. A regex is far less readable than the original grammar, because it lacks the nonterminal names that documented the meaning of each subexpression. But many programming languages have library support for regexes (and not for grammars), and regexes are much faster to match than a grammar.

The regex syntax commonly implemented in programming language libraries has a few more special characters, in addition to the ones we used above in grammars. Here are some common useful ones:

```regex
.   // matches any single character (but sometimes excluding newline, depending on the regex library)

\d  // matches any digit, same as [0-9]
\s  // matches any whitespace character, including space, tab, newline
\w  // matches any word character including underscore, same as [a-zA-Z_0-9]
```
Backslash is also used to “escape” an operator or special character so that it matches literally. Here are some of the common special characters that you need to escape:

```regex
\.  \(  \)  \*  \+  \|  \[  \]  \\
```

```regex
http://([a-z]+\.)+[a-z]+(:[0-9]+)?/
```

Another way to escape a special character is to wrap it in character-class brackets. Instead of \., we could also write [.] to match just the literal . character. Inside character-class brackets, most special characters lose their special meaning, and are simply treated literally. But characters that are special to the character-class syntax, like [, ], ^, -, and \, still need to be escaped by a backslash to be used literally.